In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import utils

import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
num_classes = 10
img_rows, img_cols = 28, 28

In [3]:
mnist = keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [4]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [5]:
def squash(x, axis=-1):
    squared_norm = keras.backend.sum(keras.backend.square(x), axis, keepdims=True) + keras.backend.epsilon()
    scale = keras.backend.sqrt(squared_norm)/(1+squared_norm)
    return scale*x

In [6]:
class digitCaps(keras.layers.Layer):
    def __init__(self,in_type=1152, in_shape=8, out_type=10, out_shape=16, routings=3, activation='squash',**kwargs):
        self.in_type = in_type
        self.in_shape = in_shape       
        self.out_type = out_type
        self.out_shape = out_shape
        self.routings = routings
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = keras.layers.Activation(activation)
        super(digitCaps,self).__init__(**kwargs)
        
    def build(self,input_shape):
        self.kernel1 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel2 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel3 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel4 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel5 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel6 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel7 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel8 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel9 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel10 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        super(digitCaps,self).build(input_shape)

    def call(self,inputs):
        type1 = tf.squeeze(tf.matmul(inputs, self.kernel1),axis=2)
        type2 = tf.squeeze(tf.matmul(inputs, self.kernel2),axis=2)
        type3 = tf.squeeze(tf.matmul(inputs, self.kernel3),axis=2)
        type4 = tf.squeeze(tf.matmul(inputs, self.kernel4),axis=2)
        type5 = tf.squeeze(tf.matmul(inputs, self.kernel5),axis=2)
        type6 = tf.squeeze(tf.matmul(inputs, self.kernel6),axis=2)
        type7 = tf.squeeze(tf.matmul(inputs, self.kernel7),axis=2)
        type8 = tf.squeeze(tf.matmul(inputs, self.kernel8),axis=2)
        type9 = tf.squeeze(tf.matmul(inputs, self.kernel9),axis=2)
        type10 = tf.squeeze(tf.matmul(inputs, self.kernel10),axis=2)
        result = tf.stack([type1,type2,type3,type4,type5,type6,type7,type8,type9,type10],1)
        B = tf.zeros_like(tf.transpose(tf.transpose(result)[1]))
        for i in range(self.routings):
            C = tf.nn.softmax(B,1)
            O = tf.einsum('ijk,ijkl->ijl', C, result)
            V = squash(O,-1)
            B = B + tf.einsum('ijk,ijlk->ijl', V, result)
        return V

In [7]:
class selfCaps(keras.layers.Layer):
    def __init__(self,in_type=1152, in_shape=8, out_type=10, out_shape=16, activation='squash',**kwargs):
        self.in_type = in_type
        self.in_shape = in_shape       
        self.out_type = out_type
        self.out_shape = out_shape
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = keras.layers.Activation(activation)
        super(selfCaps,self).__init__(**kwargs)
        
    def build(self,input_shape):
        self.kernel1 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel2 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel3 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel4 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel5 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel6 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel7 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel8 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel9 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.kernel10 = self.add_weight(shape=(self.in_type,self.in_shape,self.out_shape),initializer="uniform", trainable=True)
        self.O = self.add_weight(shape=(self.out_type,self.out_shape,self.out_shape),initializer="ones", trainable=True)
        super(selfCaps,self).build(input_shape)

    def call(self,inputs):
        type1 = tf.squeeze(tf.matmul(inputs, self.kernel1),axis=2)
        type2 = tf.squeeze(tf.matmul(inputs, self.kernel2),axis=2)
        type3 = tf.squeeze(tf.matmul(inputs, self.kernel3),axis=2)
        type4 = tf.squeeze(tf.matmul(inputs, self.kernel4),axis=2)
        type5 = tf.squeeze(tf.matmul(inputs, self.kernel5),axis=2)
        type6 = tf.squeeze(tf.matmul(inputs, self.kernel6),axis=2)
        type7 = tf.squeeze(tf.matmul(inputs, self.kernel7),axis=2)
        type8 = tf.squeeze(tf.matmul(inputs, self.kernel8),axis=2)
        type9 = tf.squeeze(tf.matmul(inputs, self.kernel9),axis=2)
        type10 = tf.squeeze(tf.matmul(inputs, self.kernel10),axis=2)
        result = tf.stack([type1,type2,type3,type4,type5,type6,type7,type8,type9,type10],1)
        V = tf.reduce_sum(result, axis=2, keepdims=True)
        V = squash(V,-1)
        V = tf.squeeze(tf.matmul(V, self.O),axis=2)
        return V

In [8]:
output = keras.layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), 2)), output_shape=(10,))

In [9]:
class mask(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(mask,self).__init__(**kwargs)
        
    def build(self,input_shape):
        super(mask,self).build(input_shape)

    def call(self,inputs):
        info = keras.backend.sqrt(keras.backend.sum(keras.backend.square(inputs), 2))
        inf = tf.argmax(info,axis=1)[1]
        out = inputs[:,inf,:]
        return out

In [ ]:
inputs = tf.keras.layers.Input(shape=(28,28,1))
C1 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(1,1), activation="relu")(inputs)
C2 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(2,2), activation="relu")(C1)
R1 = keras.layers.Reshape(target_shape=(1152,1,8))(C2)
P1 = digitCaps(in_type=1152, in_shape=8, out_type=10, out_shape=16)(R1)
out_caps = keras.layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), 2)), output_shape=(10,))(P1)
Mask = mask()(P1)
dense1 = keras.layers.Dense(512,activation="relu")(Mask)
dense2 = keras.layers.Dense(1024)(dense1)
dense3 = keras.layers.Dense(784)(dense2)
out = keras.layers.Reshape(target_shape=(28,28,1))(dense3)
model1 =  tf.keras.models.Model(inputs=inputs, outputs = [out_caps, out])
model1.summary()
model1.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
            loss=[lambda y_true,y_pred:K.mean(K.sum(y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1)),1)), 'mse'],
            metrics=['accuracy','mse'])

In [ ]:
inputs = tf.keras.layers.Input(shape=(28,28,1))
C1 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(1,1), activation="relu")(inputs)
C2 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(2,2), activation="relu")(C1)
R1 = keras.layers.Reshape(target_shape=(1152,1,8))(C2)
P1 = digitCaps(in_type=1152, in_shape=8, out_type=10, out_shape=16)(R1)
out_caps = keras.layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), 2)), output_shape=(10,))(P1)
Mask = mask()(P1)
K1 = keras.layers.Dense(1152,activation="relu")(Mask)
K2 = keras.layers.Reshape(target_shape=(6,6,32))(K1)
K3 = keras.layers.Conv2DTranspose(64,(9,9),(2,2))(K2)
K4 = keras.layers.Conv2DTranspose(1,(10,10))(K3)
model2 =  tf.keras.models.Model(inputs=inputs, outputs = [out_caps, K4])
model2.summary()
model2.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
            loss=[lambda y_true,y_pred:K.mean(K.sum(y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1)),1)), 'mse'],
            metrics=['accuracy','mse'])

In [ ]:
inputs = tf.keras.layers.Input(shape=(28,28,1))
C1 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(1,1), activation="relu")(inputs)
C2 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(2,2), activation="relu")(C1)
R1 = keras.layers.Reshape(target_shape=(1152,1,8))(C2)
P1 = selfCaps(in_type=1152, in_shape=8, out_type=10, out_shape=16)(R1)
out_caps = keras.layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), 2)), output_shape=(10,))(P1)
Mask = mask()(P1)
dense1 = keras.layers.Dense(512)(Mask)
dense2 = keras.layers.Dense(1024)(dense1)
dense3 = keras.layers.Dense(784)(dense2)
out = keras.layers.Reshape(target_shape=(28,28,1))(dense3)
model3 =  tf.keras.models.Model(inputs=inputs, outputs = [out_caps, out])
model3.summary()
model3.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
            loss=[lambda y_true,y_pred:K.mean(K.sum(y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1)),1)), 'mse'],
            metrics=['accuracy','mse'])

In [ ]:
inputs = tf.keras.layers.Input(shape=(28,28,1))
C1 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(1,1), activation="relu")(inputs)
C2 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(2,2), activation="relu")(C1)
R1 = keras.layers.Reshape(target_shape=(1152,1,8))(C2)
P1 = selfCaps(in_type=1152, in_shape=8, out_type=10, out_shape=16)(R1)
out_caps = keras.layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), 2)), output_shape=(10,))(P1)
Mask = mask()(P1)
K1 = keras.layers.Dense(1152,activation="relu")(Mask)
K2 = keras.layers.Reshape(target_shape=(6,6,32))(K1)
K3 = keras.layers.Conv2DTranspose(64,(9,9),(2,2))(K2)
K4 = keras.layers.Conv2DTranspose(1,(10,10))(K3)
model4 =  tf.keras.models.Model(inputs=inputs, outputs = [out_caps, K4])
model4.summary()
model4.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
            loss=[lambda y_true,y_pred:K.mean(K.sum(y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1)),1)), 'mse'],
            metrics=['accuracy','mse'])

In [ ]:
x_val = x_train[:12000]
y_val = y_train[:12000]
x_train = x_train[12000:]
y_train = y_train[12000:]

In [ ]:
hismodel1 = model1.fit(x_train,[y_train,x_train],epochs=20,batch_size=128,validation_data=[x_val, [y_val,x_val]])

In [ ]:
hismodel2 = model2.fit(x_train,[y_train,x_train],epochs=20,batch_size=128,validation_data=[x_val, [y_val,x_val]])

In [ ]:
hismodel3 = model3.fit(x_train,[y_train,x_train],epochs=20,batch_size=128,validation_data=[x_val, [y_val,x_val]])

In [ ]:
hismodel4 = model4.fit(x_train,[y_train,x_train],epochs=20,batch_size=128,validation_data=[x_val, [y_val,x_val]])

In [ ]:
import matplotlib.pyplot as plt
loss = hismodel1.history['loss']
val_loss =hismodel1.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig("result1.png")

In [ ]:
acc = hismodel1.history['lambda_1_accuracy']
val_acc = hismodel1.history['val_lambda_1_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("resultq1.png")

In [ ]:
import matplotlib.pyplot as plt
loss = hismodel2.history['loss']
val_loss =hismodel2.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig("result2.png")

In [ ]:
acc = hismodel2.history['lambda_2_accuracy']
val_acc = hismodel2.history['val_lambda_2_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("resultq2.png")

In [ ]:
import matplotlib.pyplot as plt
loss = hismodel3.history['loss']
val_loss =hismodel3.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig("result3.png")

In [ ]:
acc = hismodel3.history['lambda_3_accuracy']
val_acc = hismodel3.history['val_lambda_3_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("resultq3.png")

In [ ]:
import matplotlib.pyplot as plt
loss = hismodel4.history['loss']
val_loss =hismodel4.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig("result4.png")

In [ ]:
acc = hismodel4.history['lambda_4_accuracy']
val_acc = hismodel4.history['val_lambda_4_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("resultq4.png")

In [ ]:
inputs = tf.keras.layers.Input(shape=(28,28,1))
C1 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(1,1), activation="relu")(inputs)
C2 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(2,2), activation="relu")(C1)
R1 = keras.layers.Reshape(target_shape=(1152,1,8))(C2)
P1 = digitCaps(in_type=1152, in_shape=8, out_type=10, out_shape=16)(R1)
out_caps = keras.layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), 2)), output_shape=(10,))(P1)
Mask = mask()(P1)
dense1 = keras.layers.Dense(512,activation="relu")(Mask)
dense2 = keras.layers.Dense(1024)(dense1)
dense3 = keras.layers.Dense(784)(dense2)
out = keras.layers.Reshape(target_shape=(28,28,1))(dense3)
model5 =  tf.keras.models.Model(inputs=inputs, outputs = [out_caps, out])
model5.summary()
model5.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
            loss=[lambda y_true,y_pred:K.mean(K.sum(y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1)),1)), 'mse'],
            metrics=['accuracy','mse'])

In [ ]:
inputs = tf.keras.layers.Input(shape=(28,28,1))
C1 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(1,1), activation="relu")(inputs)
C2 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(2,2), activation="relu")(C1)
R1 = keras.layers.Reshape(target_shape=(1152,1,8))(C2)
P1 = digitCaps(in_type=1152, in_shape=8, out_type=10, out_shape=16)(R1)
out_caps = keras.layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), 2)), output_shape=(10,))(P1)
Mask = mask()(P1)
K1 = keras.layers.Dense(1152,activation="relu")(Mask)
K2 = keras.layers.Reshape(target_shape=(6,6,32))(K1)
K3 = keras.layers.Conv2DTranspose(64,(9,9),(2,2))(K2)
K4 = keras.layers.Conv2DTranspose(1,(10,10))(K3)
model6 =  tf.keras.models.Model(inputs=inputs, outputs = [out_caps, K4])
model6.summary()
model6.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
            loss=[lambda y_true,y_pred:K.mean(K.sum(y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1)),1)), 'mse'],
            metrics=['accuracy','mse'])

In [ ]:
inputs = tf.keras.layers.Input(shape=(28,28,1))
C1 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(1,1), activation="relu")(inputs)
C2 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(2,2), activation="relu")(C1)
R1 = keras.layers.Reshape(target_shape=(1152,1,8))(C2)
P1 = selfCaps(in_type=1152, in_shape=8, out_type=10, out_shape=16)(R1)
out_caps = keras.layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), 2)), output_shape=(10,))(P1)
Mask = mask()(P1)
dense1 = keras.layers.Dense(512)(Mask)
dense2 = keras.layers.Dense(1024)(dense1)
dense3 = keras.layers.Dense(784)(dense2)
out = keras.layers.Reshape(target_shape=(28,28,1))(dense3)
model7 =  tf.keras.models.Model(inputs=inputs, outputs = [out_caps, out])
model7.summary()
model7.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
            loss=[lambda y_true,y_pred:K.mean(K.sum(y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1)),1)), 'mse'],
            metrics=['accuracy','mse'])

In [ ]:
inputs = tf.keras.layers.Input(shape=(28,28,1))
C1 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(1,1), activation="relu")(inputs)
C2 = keras.layers.Conv2D(filters=256, kernel_size=(9,9), strides=(2,2), activation="relu")(C1)
R1 = keras.layers.Reshape(target_shape=(1152,1,8))(C2)
P1 = selfCaps(in_type=1152, in_shape=8, out_type=10, out_shape=16)(R1)
out_caps = keras.layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), 2)), output_shape=(10,))(P1)
Mask = mask()(P1)
K1 = keras.layers.Dense(1152,activation="relu")(Mask)
K2 = keras.layers.Reshape(target_shape=(6,6,32))(K1)
K3 = keras.layers.Conv2DTranspose(64,(9,9),(2,2))(K2)
K4 = keras.layers.Conv2DTranspose(1,(10,10))(K3)
model8 =  tf.keras.models.Model(inputs=inputs, outputs = [out_caps, K4])
model8.summary()
model8.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
            loss=[lambda y_true,y_pred:K.mean(K.sum(y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1)),1)), 'mse'],
            metrics=['accuracy','mse'])

In [ ]:
mnist = keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [ ]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [ ]:
hismodel5 = model5.fit(x_train,[y_train,x_train],epochs=4,batch_size=128)

In [ ]:
hismodel6 = model6.fit(x_train,[y_train,x_train],epochs=4,batch_size=128)

In [ ]:
hismodel7 = model7.fit(x_train,[y_train,x_train],epochs=4,batch_size=128)

In [ ]:
hismodel8 = model8.fit(x_train,[y_train,x_train],epochs=4,batch_size=128)

In [ ]:
model5.evaluate(x_test,[y_test,x_test])

In [ ]:
model6.evaluate(x_test,[y_test,x_test])

In [ ]:
model7.evaluate(x_test,[y_test,x_test])

In [ ]:
model8.evaluate(x_test,[y_test,x_test])

In [ ]:
import matplotlib.pyplot as plt

x1 = model5.predict(x_test)[1]
x2 = model6.predict(x_test)[1]
x3 = model7.predict(x_test)[1]
x4 = model8.predict(x_test)[1]

In [ ]:
num=11
y1 = x1[num].reshape(28,28)
plt.imshow(y1)

In [ ]:

y2 = x2[num].reshape(28,28)
plt.imshow(y2)

In [ ]:

y3 = x3[num].reshape(28,28)
plt.imshow(y3)

In [ ]:

y4 = x4[num].reshape(28,28)
plt.imshow(y4)

In [ ]:
plt.imshow(x_test[num].reshape(28,28))